# Data Preprocessing
## Step 1: Load and Sample the Dataset
- Load the US Accidents dataset.
- Sample to 10,000 rows for efficiency.
- Display basic information.

In [1]:
import pandas as pd

# Parameters
chunk_size = 100000  # Read 100,000 rows at a time
target_sample_size = 10000  # Total rows we want
sampled_chunks = []  # To store sampled data

# Calculate sampling fraction per chunk
total_rows_approx = 7700000  # Approximate total rows in the dataset
chunks_approx = total_rows_approx // chunk_size
rows_per_chunk = target_sample_size // chunks_approx

print(f"Approximate chunks: {chunks_approx}")
print(f"Rows to sample per chunk: {rows_per_chunk}")

# Read the dataset in chunks and sample
for chunk in pd.read_csv('../data/US_Accidents_March23.csv', chunksize=chunk_size, low_memory=False):
    # Sample rows from the chunk
    if len(chunk) >= rows_per_chunk:
        sampled_chunk = chunk.sample(n=rows_per_chunk, random_state=42)
    else:
        sampled_chunk = chunk  # If chunk is smaller than rows_per_chunk, take all rows
    sampled_chunks.append(sampled_chunk)

    # Print progress
    print(f"Processed chunk with {len(chunk)} rows, sampled {len(sampled_chunk)} rows")

    # Stop if we have enough rows
    if sum(len(c) for c in sampled_chunks) >= target_sample_size:
        break

# Combine sampled chunks
df_sampled = pd.concat(sampled_chunks, ignore_index=True)

# Trim to exact target size
if len(df_sampled) > target_sample_size:
    df_sampled = df_sampled.sample(n=target_sample_size, random_state=42)

# Save the sampled dataset
df_sampled.to_csv('../data/usa_traffic_accidents_sampled.csv', index=False)
print("Sampled dataset saved to data/usa_traffic_accidents_sampled.csv")

# Load the sampled dataset
df = pd.read_csv('../data/usa_traffic_accidents_sampled.csv')

# Display info
print("Sampled Dataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())
print("\nColumns:")
print(df.columns.tolist())

Approximate chunks: 77
Rows to sample per chunk: 129
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 rows, sampled 129 rows
Processed chunk with 100000 r

## Step 2: Clean and Preprocess
- Load the sampled dataset.
- Handle missing values.
- Convert data types and extract time features.
- Infer lighting condition from Sunrise_Sunset.
- Save the cleaned dataset.

In [7]:
import pandas as pd

# Load the sampled dataset
df = pd.read_csv('../data/usa_traffic_accidents_sampled.csv')

# Check missing values
print("Missing values (%):")
print((df.isnull().sum() / len(df) * 100).sort_values(ascending=False))

# Drop rows with missing critical columns
df = df.dropna(subset=['Start_Time', 'Start_Lat', 'Start_Lng', 'Weather_Condition'])

# Infer Road_Condition from Weather_Condition and Precipitation(in)
def infer_road_condition(row):
    weather = str(row['Weather_Condition']).lower()
    precipitation = row['Precipitation(in)'] if pd.notnull(row['Precipitation(in)']) else 0
    
    if 'rain' in weather or 'snow' in weather or 'ice' in weather or 'sleet' in weather or precipitation > 0:
        return 'Wet'
    elif 'clear' in weather or 'cloudy' in weather or 'fair' in weather:
        return 'Dry'
    else:
        return 'Unknown'

df['Road_Condition'] = df.apply(infer_road_condition, axis=1)

# Fill missing categorical columns
df['Sunrise_Sunset'] = df['Sunrise_Sunset'].fillna('Unknown')
df['City'] = df['City'].fillna('Unknown')
df['State'] = df['State'].fillna('Unknown')

# Convert Start_Time to datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

# Extract time features
df['Day_of_Week'] = df['Start_Time'].dt.day_name()
df['Hour'] = df['Start_Time'].dt.hour
df['Month'] = df['Start_Time'].dt.month_name()

# Drop invalid datetime rows
df = df.dropna(subset=['Start_Time'])

# Infer Light_Condition from Sunrise_Sunset
df['Light_Condition'] = df['Sunrise_Sunset'].map({'Day': 'Daylight', 'Night': 'Dark', 'Unknown': 'Unknown'})

# Save cleaned dataset
df.to_csv('../data/usa_traffic_accidents_cleaned.csv', index=False)
print("Cleaned dataset saved to data/usa_traffic_accidents_cleaned.csv")

# Display info
print("Cleaned Dataset Info:")
print(df.info())
print("\nSample features:")
print(df[['Start_Time', 'Day_of_Week', 'Hour', 'Month', 'Weather_Condition', 'Road_Condition', 'Light_Condition']].head())

Missing values (%):
End_Lng                  43.65
End_Lat                  43.65
Precipitation(in)        28.20
Wind_Chill(F)            25.28
Wind_Speed(mph)           7.37
Visibility(mi)            2.43
Wind_Direction            2.42
Humidity(%)               2.39
Weather_Condition         2.37
Temperature(F)            2.27
Pressure(in)              2.00
Weather_Timestamp         1.63
Sunrise_Sunset            0.33
Airport_Code              0.33
Astronomical_Twilight     0.33
Nautical_Twilight         0.33
Civil_Twilight            0.33
Street                    0.17
Timezone                  0.09
Zipcode                   0.01
Description               0.00
City                      0.00
Source                    0.00
ID                        0.00
Severity                  0.00
Distance(mi)              0.00
Start_Lng                 0.00
Start_Time                0.00
End_Time                  0.00
Start_Lat                 0.00
County                    0.00
Amenity            